In [1]:
import pandas as pd
import numpy as np
import datatable as dt
import gc

IGNORE_COLS = ['row_id', 'f_30', 'f_63', 'f_74', 'f_84', 'f_120', 'f_124', 'f_170', 'f_291', 'f_0', 
               'f_1', 'f_4', 'f_6', 'f_7', 'f_10', 'f_11', 'f_12', 'f_13', 'f_14', 'f_18', 'f_19', 
               'f_20', 'f_22', 'f_24', 'f_27', 'f_28', 'f_30', 'f_32', 'f_34', 'f_35', 'f_37', 'f_39', 
               'f_40', 'f_46', 'f_48', 'f_49', 'f_50', 'f_51', 'f_53', 'f_56', 'f_57', 'f_60', 'f_61', 
               'f_62', 'f_63', 'f_66', 'f_67', 'f_70', 'f_73', 'f_74', 'f_75', 'f_81', 'f_84', 'f_86', 
               'f_89', 'f_95', 'f_97', 'f_99', 'f_101', 'f_102', 'f_107', 'f_108', 'f_110', 'f_111', 
               'f_115', 'f_117', 'f_120', 'f_122', 'f_123', 'f_124', 'f_126', 'f_127', 'f_128', 'f_129', 
               'f_130', 'f_131', 'f_138', 'f_140', 'f_141', 'f_142', 'f_143', 'f_148', 'f_149', 'f_151', 
               'f_152', 'f_154', 'f_156', 'f_157', 'f_159', 'f_161', 'f_164', 'f_166', 'f_170', 'f_171', 
               'f_175', 'f_176', 'f_178', 'f_180', 'f_182', 'f_183', 'f_186', 'f_189', 'f_191', 'f_195', 
               'f_196', 'f_198', 'f_199', 'f_200', 'f_201', 'f_202', 'f_204', 'f_205', 'f_208', 'f_211', 
               'f_213', 'f_215', 'f_217', 'f_218', 'f_220', 'f_221', 'f_222', 'f_223', 'f_227', 'f_228', 
               'f_229', 'f_235', 'f_236', 'f_240', 'f_241', 'f_242', 'f_244', 'f_248', 'f_253', 'f_254', 
               'f_258', 'f_261', 'f_262', 'f_267', 'f_269', 'f_272', 'f_275', 'f_276', 'f_278', 'f_279', 
               'f_282', 'f_283', 'f_284', 'f_285', 'f_288', 'f_289', 'f_291', 'f_292', 'f_293', 'f_294', 
               'f_296', 'f_298', 'f_299', 'f_262']

basic_cols = ['time_id', 'investment_id', 'target']
num_feat = 300
features = [f'f_{i}' for i in range(num_feat)]
cols = basic_cols + features
selected_cols = []
for c in cols:
    if c in IGNORE_COLS:
        continue
    selected_cols.append(c)

#train = pd.read_csv('../input/ubiquant-market-prediction/train.csv', skiprows=lines2skip)
train = dt.fread('../input/ubiquant-market-prediction/train.csv').to_pandas()
train = train[selected_cols]
test = pd.read_csv('../input/ubiquant-market-prediction/example_test.csv')
sub = pd.read_csv('../input/ubiquant-market-prediction/example_sample_submission.csv')

In [2]:
train.dtypes

time_id            int32
investment_id      int32
target           float64
f_2              float64
f_3              float64
                  ...   
f_286            float64
f_287            float64
f_290            float64
f_295            float64
f_297            float64
Length: 155, dtype: object

In [3]:
train.head()

,time_id,investment_id,target,f_2,f_3,f_5,f_8,f_9,f_15,f_16,...,f_273,f_274,f_277,f_280,f_281,f_286,f_287,f_290,f_295,f_297
0,0,1,-0.300875,-0.402206,0.378386,-0.413469,0.114809,-2.012777,-0.267310,-0.574423,...,1.281245,0.258715,-0.324677,-0.025610,-0.006259,0.301659,0.157470,0.366028,-0.086764,-1.044826
1,0,2,-0.231040,0.742368,-0.616673,1.771210,0.114809,-0.219201,-0.010217,-0.681523,...,-0.650803,0.851905,0.298990,0.532229,0.226693,-0.557502,-0.875265,-0.154193,-0.387617,-0.929529
2,0,6,0.568807,0.567806,-0.607963,-1.083155,0.114809,-1.035376,-0.451820,-0.467322,...,0.098910,-0.725177,0.260281,-0.315315,0.150139,-0.147156,-0.087518,-0.138020,-0.219097,-0.612428
3,0,7,-1.064780,1.874606,-0.606346,-0.815737,0.114809,-1.176410,-1.121598,-0.681523,...,-0.817650,-0.624180,-1.796107,-0.640443,2.324936,-0.557502,-0.986692,0.382201,-0.609113,-0.783423
4,0,8,-0.531940,2.330030,-0.583422,-0.742814,0.114809,-0.005858,-0.829021,-0.826026,...,-1.332959,-0.767390,-1.465071,-0.781270,-1.918721,-0.519032,-1.221407,-0.170365,-0.588445,0.753279


In [4]:
sub.head()

,row_id,time_id,target
0,1220_1,1220,0
1,1220_2,1220,0
2,1221_0,1221,0
3,1221_1,1221,0
4,1221_2,1221,0


In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor 
import lightgbm 
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import gc

from sklearn.metrics import r2_score
from joblib import dump, load
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

import optuna

X = train.drop(['target'], axis=1)
y = train[['target']]
del train
gc.collect();

r1 = LinearRegression()
r2 = RandomForestRegressor()
r3 = lightgbm.LGBMRegressor(n_estimators=700, max_depth=13, learning_rate=0.04, subsample=0.9, colsample_bytree=0.7, random_state=1700)

In [6]:
ensemble = VotingRegressor([('lr', r1), ('lgbm', r3)], weights=[0.985, 0.974])
ensemble.fit(X, y)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


VotingRegressor(estimators=[('lr', LinearRegression()),
                            ('lgbm',
                             LGBMRegressor(colsample_bytree=0.7,
                                           learning_rate=0.04, max_depth=13,
                                           n_estimators=700, random_state=1700,
                                           subsample=0.9))],
                weights=[0.985, 0.974])

# Modeling

# Improve VotingRegressor / Give a weighted system using the adjusted r2_score of each model. 
Adjusted r2_score might be appropriate to evaluate the regression models because the dataset incluedes the large number of features (f_{i}), and adjusted r2_score gives the penalty to the number of features to prevent overfitting problem. Since the test dataset is totally different from training dataset, solving overfitting problem would be important 

r1 = LinearRegression()
r2 = Ridge()
r3 = lightgbm.LGBMRegressor(n_estimators=700,
                            max_depth=11,
                            learning_rate=0.05,
                            subsample=0.9,
                            colsample_bytree=0.7,
                            random_state=2022)

def get_models(): 
    models = list()
    models.append(('lr', r1)) 
    models.append(('lgbm', r3))
    return models

def evaluate_models(models, X_train, X_val, y_train, y_val): 
    
   scores = list()
    for name, model in models: 
        model.fit(X_train, y_train)
        prediction = model.predict(X_val)
        MSE = 100 / (mean_squared_error(y_val, prediction))
        scores.append(MSE)
        
   return scores

In [7]:
gc.collect();
try:
    selected_cols.remove('target')
except:
    pass
preds = ensemble.predict(test[selected_cols])
preds

array([-0.10347985, -0.03565308, -0.04449298, -0.04599277, -0.04966679,
       -0.02660141, -0.04647437, -0.07789554, -0.0369379 ])

In [8]:
import ubiquant
env = ubiquant.make_env()
iter_test = env.iter_test()

In [9]:
for (test_df, sample_prediction_df) in iter_test:
    row_id = test_df['row_id'].str.split('_', expand=True)
    test_df['time_id'] = row_id[0]
    test_df['time_id'] = test_df['time_id'].astype(np.int32)
    preds = ensemble.predict(test_df[selected_cols])  # make your predictions here
    sample_prediction_df['target'] = preds
    
    env.predict(sample_prediction_df)   # register your predictions

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
